# EHR

*Classes*
   - Patient
     - CrCl Calculation Display
   - LabReference

In [3]:
import sympy as sp
import random
from IPython.display import Markdown
import pandas as pd

## Patient (class)

   - CrCl (function)

In [10]:
class Patient:
   def __init__(self, name, weight, age, SerCreat):
      self.name = name
      self.id = random.randint(1000000,10000000)
      self.weight = weight
      self.age = age
      self.SerCreat = SerCreat
      self.CrCl_value = CrCl = ((140-self.age) * self.weight) / (72 * self.SerCreat)
   
   def as_dict (self):
      return {"name": self.name, "id": self.id, "weight":self.weight, "age":self.age, "SerumCreatinine": self.SerCreat, "CrCl": self.CrCl_value}
   
   @property
   def CrCl (self):
      name = self.name
      age = self.age 
      weight = self.weight
      scr = self.SerCreat
      
      a , w , Cr = sp.symbols("a_yr W_kg Cr_Ser")
      CrCl = ((140-a) * w) / (72 * Cr)
      
      md = Markdown("$\large CrCl_{" + name + "} = " + sp.Subs(CrCl,(a,w,Cr),(age,weight,scr))._repr_latex_()[1:]+ "  $\large = \\boxed {" + str(round(CrCl.subs({a:age,w:weight,Cr:scr}),1)) + "}$")
      
      return md

   def __repr__(self):
      return f'Name: {self.name}\n\tID:{self.id}\n\t{self.weight}kg\n\t{self.age}yr.of age\n\n\tCr(serum): {self.SerCreat}\n\t{self.CrCl_value}'

In [13]:
Patient("Josh Steinbecker",68,27, 0.888).CrCl

$\large CrCl_{Josh Steinbecker} = \displaystyle \left. \frac{W_{kg} \left(140 - a_{yr}\right)}{72 Cr_{Ser}} \right|_{\substack{ a_{yr}=27\\ W_{kg}=68\\ Cr_{Ser}=0.888 }}$  $\large = \boxed {120.2}$

In [17]:
Patient("Bob", 79, 59, 1.629).CrCl

$\large CrCl_{Bob} = \displaystyle \left. \frac{W_{kg} \left(140 - a_{yr}\right)}{72 Cr_{Ser}} \right|_{\substack{ a_{yr}=59\\ W_{kg}=79\\ Cr_{Ser}=1.629 }}$  $\large = \boxed {54.6}$

## LabReference (class)
```py
   >>> refID
   >>> Name
   >>> Low
   >>> High
   >>> Unit *
   >>> Class *
```

In [7]:
class LabReference:
   refID:      str
   name:       str
   lowbound:   float
   highbound:  float
   unit:       str
   def __init__(self,refID,name,low,high,Unit=None,Class="Chem"):
      self.refID = refID
      self.Class = Class
      self.name = name
      self.lowbound = low
      self.highbound = high
      self.unit = Unit
   def __repr__(self):
      return f'[{self.name} ({self.lowbound} - {self.highbound})]'
   def as_dict (self):
      return {"id": self.refID, "class":self.Class,"name":self.name, "low": self.lowbound, "high": self.highbound, "unit":self.unit }

In [32]:
NA = LabReference("Na","Sodium(serum)",131,145,"mg/dL")
K = LabReference("K","Potassium(serum)",3.1,5.1,"mg/dL")
CA = LabReference("Ca","Calcium(serum)",8.1,10,"mg/dL")
CO3 = LabReference("CO3","Bicarbonate",10,40,"mg/dL")
GLU = LabReference("GLU","Glucose",90,100,"mg/dL")
VANC_TROUGH = LabReference("Vanc[T]","Vancomycin(serum)(trough)",11,20,"mg/dL","ABX")
PLT = LabReference("PLT","Platelet",450000,None,"/dL","Hemo")

ALLLABS = [NA,K,CA,CO3,VANC_TROUGH,GLU,PLT]
ALLLABS


[[Sodium(serum) (131 - 145)],
 [Potassium(serum) (3.1 - 5.1)],
 [Calcium(serum) (8.1 - 10)],
 [Bicarbonate (10 - 40)],
 [Vancomycin(serum)(trough) (11 - 20)],
 [Glucose (90 - 100)],
 [Platelet (450000 - None)]]

In [33]:

df = pd.DataFrame([x.as_dict() for x in ALLLABS])
df.set_index(["class","id"]).sort_index()

name       low   high   unit
class id                                                        
ABX   Vanc[T]  Vancomycin(serum)(trough)      11.0   20.0  mg/dL
Chem  CO3                    Bicarbonate      10.0   40.0  mg/dL
      Ca                  Calcium(serum)       8.1   10.0  mg/dL
      GLU                        Glucose      90.0  100.0  mg/dL
      K                 Potassium(serum)       3.1    5.1  mg/dL
      Na                   Sodium(serum)     131.0  145.0  mg/dL
Hemo  PLT                       Platelet  450000.0    NaN    /dL

In [37]:
df.iloc[3], df.iloc[6]

(id               CO3
 class           Chem
 name     Bicarbonate
 low             10.0
 high            40.0
 unit           mg/dL
 Name: 3, dtype: object,
 id            PLT
 class        Hemo
 name     Platelet
 low      450000.0
 high          NaN
 unit          /dL
 Name: 6, dtype: object)